# Update names with translations from OEC

In [60]:
import requests
import pandas as pd
import sys
import configparser
from sqlalchemy import text

In [22]:
config = configparser.ConfigParser()
config.read_file(open('../../settings.ini'))

sys.path.insert(0, config.get('PATHS','libs_path'))
engine_path = config.get('DATABASE','engine_path')

import postgres
from sqlalchemy import create_engine
engine = create_engine(engine_path)
db = postgres.PostgresDriver(engine)

In [33]:
oec_xlations = pd.DataFrame(requests.get("https://atlas.media.mit.edu/attr/hs92/es/").json()['data'])
oec_6 = oec_xlations[(oec_xlations.id.str.len() == 6)]

In [23]:
datachile_hs = pd.read_sql("SELECT * FROM economy.hs_levels", engine)

In [30]:
datachile_hs.loc[:, 'oec_id_level3'] = datachile_hs.apply(lambda d: d.level0 + d.level2, axis=1)

In [41]:
merged = datachile_hs.merge(oec_6, how='inner', left_on='oec_id_level3', right_on='id')


In [56]:
merged['name']

0                        Válvulas
1                        Válvulas
2                        Válvulas
3                        Válvulas
4                        Válvulas
5                        Válvulas
6                           Avena
7                        Caballos
8                        Caballos
9                        Caballos
10                  Los aldehídos
11                  Los aldehídos
12                  Los aldehídos
13                  Los aldehídos
14                  Los aldehídos
15                  Los aldehídos
16                  Los aldehídos
17                  Los aldehídos
18                  Los aldehídos
19                  Los aldehídos
20                  Los aldehídos
21                  Los aldehídos
22                       Raw Silk
23               Algodón en bruto
24                         Bovino
25                         Bovino
26        Hidrocarburos acíclicos
27        Hidrocarburos acíclicos
28        Hidrocarburos acíclicos
29        Hidr

In [ ]:
#merged[['level0', 'level0_name', 'es_level0_description', 'level1', 'level1_name', 'level2', 'level2_name', 'name']]
engine.begin()
for _, r in merged.iterrows():
#    print(r['name'])
    engine.execute(text("UPDATE economy.hs_levels SET es_level2_description = :name WHERE :id = (level0 || level2)"), name=r['name'], id=r['oec_id_level3'])
    #engine.execute(, r['name'], r['oec_id_level3'])
engine.commit()
